Части слова я предсказывала при помощи логистической регрессии, на кросс валидации был довольно хороший результат - 96%.

Для начальной формы я завела отдельные массивы с окончаниями и куда они переходят для каждой части речи, убрала повторяющиеся и отсортировала по частоте встречаемости. Далее проходила по каждому слову в тестовой выборке и искала подходящее окончание отдельно для каждой части речи.

К сожалению, такой алгори

In [588]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

train = pd.read_csv("task2_lemmas_train.txt", header=None, names=['word','y'])
test = pd.read_csv("task2_lemmas_test.txt", header=None, names=['word'])
linear_ans_example = pd.read_csv("task2_lemmas_sample_submission.txt")

In [589]:
X = train
X['part'] = [s[-1] for s in train['y']]
X['y'] = [s[:-2] for s in train['y']]

In [590]:
X = X.drop(X.index[[0]])
X.head()

,word,y,part
1,vergognerete,vergognare,V
2,amnistiavate,amnistiare,V
3,menomazione,menomazione,N
4,sfaldavamo,sfaldare,V
5,sfodererei,sfoderare,V


In [591]:
test = test.drop(test.index[[0]])
test.head()

,word
1,gettonan
2,incidentali
3,involtino
4,lievi
5,comunistizzasse


In [500]:
ngram_vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(1,8), 
                                   min_df=1)
counts = ngram_vectorizer.fit_transform(X['word'])
len(ngram_vectorizer.get_feature_names())

491908

In [501]:
algo = LogisticRegression(penalty='l2')
algo.fit(counts, X['part'])
cv=StratifiedKFold(shuffle = True)

In [502]:
print (cross_val_score(algo, counts, X['part']).mean())

0.962710749537


In [503]:
matrix = ngram_vectorizer.transform(test['word'])

In [504]:
predictions = algo.predict(matrix)

In [605]:
from itertools import groupby

def sortByLength(inputStr):
    return len(inputStr[0])

def sortByFrequency(inputStr):
    return inputStr[2]

def get_endings(array):
    endings = []
    for index, row in array.iterrows():
        word = row['word']
        inf = row['y']
        for i in range(1, len(inf)):
            if (word.find(inf[:-i]) == 0):
                endings.append((word[len(inf[:-i]):], inf[len(inf) - i:]))
                break
    endings.sort(key=sortByLength, reverse=True)
    frequencies = [len(list(group)) for key, group in groupby(endings)]
    endings = list(set(endings))
    end = [[e[0], e[1], 0] for e in endings]
    for i in range(len(end)):
        end[i][2] = frequencies[i]
    end.sort(key=sortByFrequency, reverse=True)
    return end

In [606]:
verbs = X[X['part'] == 'V']
verbs[:10]

,word,y,part
1,vergognerete,vergognare,V
2,amnistiavate,amnistiare,V
4,sfaldavamo,sfaldare,V
5,sfodererei,sfoderare,V
6,ascondesti,ascondere,V
7,edifichereste,edificare,V
8,maschieran,maschiare,V
9,transennasser,transennare,V
10,computando,computare,V
11,accudisser,accudire,V


In [607]:
verbs_endings = get_endings(verbs)

In [608]:
verbs_endings[:5]

[['a', 'ire', 90],
 ['ieni', 'enire', 42],
 ['cciamo', 're', 29],
 ['rresti', 'nere', 24],
 ['ide', 'edere', 21]]

In [509]:
adv = X[X['part'] == 'A']
adv[:10]

,word,y,part
38,balenieri,baleniero,A
49,osanti,osare,A
50,rialzata,rialzato,A
56,rivaleggiante,rivaleggiare,A
62,cospiranti,cospirare,A
82,turbolenta,turbolento,A
84,vaticinante,vaticinare,A
90,secco,secco,A
109,quintupli,quintuplo,A
120,costiera,costiero,A


In [592]:
adv_endings = get_endings(adv)

In [593]:
adv_endings[:5]

[['nenti', 'rre', 4],
 ['centi', 'rre', 3],
 ['ù', 'ù', 3],
 ['nte', 're', 2],
 ['s', 's', 2]]

In [594]:
noun = X[X['part'] == 'N']
noun[:10]

,word,y,part
3,menomazione,menomazione,N
44,cafè,cafè,N
57,magnaccia,magnaccia,N
64,crac,crac,N
66,baionetta,baionetta,N
144,furbata,furbata,N
149,distillazioni,distillazione,N
182,demagoghi,demagogo,N
189,raccoglimenti,raccoglimento,N
194,fola,fola,N


In [595]:
noun_endings = get_endings(noun)

In [596]:
noun_endings[:5]

[['c', 'c', 4],
 ['i-chiave', 'o-chiave', 4],
 ['w', 'w', 4],
 ['a', 'o', 3],
 ['ì', 'ì', 3]]

In [597]:
test['part'] = predictions

In [598]:
test.head()

,word,part
1,gettonan,V
2,incidentali,A
3,involtino,V
4,lievi,N
5,comunistizzasse,V


In [599]:
for index, row in test.iterrows():
    word = row['word']
    if (row['part'] == 'V'):
        for i in verbs_endings:
            if (word[len(word)-len(i[0]):] == i[0]):
                test.loc[index, 'word'] = word[:len(word)-len(i[0])] + i[1]
                break
    elif (row['part'] == 'A'):
        for i in adv_endings:
            if (word[len(word)-len(i[0]):] == i[0]):
                test.loc[index, 'word'] = word[:len(word)-len(i[0])] + i[1]
                break
    else:
        for i in noun_endings:
            if (word[len(word)-len(i[0]):] == i[0]):
                test.loc[index, 'word'] = word[:len(word)-len(i[0])] + i[1]
                break

In [600]:
test[:10]

,word,part
1,gettonanre,V
2,incidentalo,A
3,involtinore,V
4,lieve,N
5,comunistizzasrre,V
6,vidimerebbere,V
7,imbrodanre,V
8,strillarre,V
9,cifrandere,V
10,compassavanore,V


In [580]:
lemma = test

In [581]:
lemma['Id'] = np.arange(1, 29662)
lemma['Category'] = lemma['word'] + '+' + lemma['part']
lemma = lemma.drop(['word', 'part'], axis=1)
lemma.head()

,Id,Category
1,1,gettonanre+V
2,2,incidentalo+A
3,3,involtinore+V
4,4,lieve+N
5,5,comunistizzasrre+V


In [582]:
lemma.to_csv("task2_lemmas_sample_submission.txt", sep=',', index=False)